# Notes on analysis of spatiotemporal dyadic interactions

## Who approaches who?
Frequency interactions are associated with phases of spatiotemporal proximity.
To understand, which of the two individuals is the approaching individual, we can compare the velocities of individuals during phases of decreasing distance. The individual that travels with a higher velocity, must be the approaching individual. The approacher is given by:

$
\text{ID}_{appr.} =
\begin{cases}
\text{ID}_{1},&\text{for} \ v_1 > v_2, \ \frac{d}{dx} < 0\\[2ex]
\text{ID}_{2},&\text{for} \ v_2 > v_1, \ \frac{d}{dx} < 0
\end{cases}
$

Conversely, if the distance becomes larger, the individual with the greater velocity is a retreater:

$
\text{ID}_{retr.} =
\begin{cases}
\text{ID}_{1},&\text{for} \ v_1 > v_2, \ \frac{d}{dx} > 0\\[2ex]
\text{ID}_{2},&\text{for} \ v_2 > v_1, \ \frac{d}{dx} > 0
\end{cases}
$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gridtools as gt
import pandas as pd
from plotstyle import PlotStyle
%matplotlib qt

In [2]:
path = "/home/weygoldt/Data/uni/efish/output/2016-04-20-18_49/"
grid = gt.GridTracks(path, finespec=False, verbose=False)
events = pd.read_csv(path+"events.csv")
idx = 8
dyad = gt.Dyad(grid, [events.id1[idx], events.id2[idx]])

v1 = gt.utils.velocity2d(dyad.times, dyad.xpos_smth_id1, dyad.ypos_smth_id1)
v2 = gt.utils.velocity2d(dyad.times, dyad.xpos_smth_id2, dyad.ypos_smth_id2)

[ GridTracks.__init__ ] No grid metadata found in directory /home/weygoldt/Data/uni/efish/output/2016-04-20-18_49/


In [3]:
ddpos = np.diff(dyad.dpos)/0.3

fig, ax = plt.subplots(5,1, sharex=True)
ax[0].plot(dyad.times, dyad.dfund)
ax[1].plot(dyad.times, dyad.dpos)
ax[2].plot(dyad.times[1:], ddpos)
ax[2].axhline(0, color='black', linewidth=1, linestyle="dashed")
ax[3].plot(dyad.times, v1)
ax[4].plot(dyad.times, v2)

ax[0].set_title("frequency difference")
ax[1].set_title("spatial difference")
ax[2].set_title("relative velocity")
ax[3].set_title("velocity id1")
ax[4].set_title("velocity id2")


Text(0.5, 1.0, 'velocity id2')

Compute relative velocity

In [4]:
dt = np.array([x - x0 for x0, x in zip(dyad.times, dyad.times[2:])])
dx = np.array([(x2 - x1) + (x1 - x0) for x0, x1, x2 in zip(dyad.dpos, dyad.dpos[1:], dyad.dpos[2:])])
vr = dx/dt
vr = gt.utils.nanpad(vr, position="center", padlen=1)

print(f"Lenght relative veloc: {len(vr)}")
print(f"Lenght time: {len(dyad.times)}")
plt.plot(dyad.times, vr)

Lenght relative veloc: 145075
Lenght time: 145075


Extract phases of approaches and phases of retreats

In [5]:
s = PlotStyle()
ylims = s.lims(dyad.fund_id1, dyad.fund_id2)

status1 = np.full(len(vr), np.nan, dtype=float)
status2 = np.full(len(vr), np.nan, dtype=float)
for i, a in enumerate(vr):
    
    # find retreats, where vr is positive (i.e. distance increases)
    # if vr[i] > 0:
    #     if v1[i] > v2[i]:
    #         status1[i] = 0
    #     elif v1[i] < v2[i]:
    #         status2[i] = 0

    # find approaches, where vr is negative (i.e. distance decreases)
    if vr[i] < 0:
        if v1[i] > v2[i]:
            status1[i] = 1
        elif v1[i] < v2[i]:
            status2[i] = 1

fig, ax1 = plt.subplots(sharex=True)

ax1.plot(dyad.times, dyad.fund_id1)
ax1.plot(dyad.times, dyad.fund_id2)
ax1.plot(dyad.times, status1+(ylims[0]-4), "|", color="tab:blue")
ax1.plot(dyad.times, status2+(ylims[0]-4), "|", color="tab:orange")
ax1.set_ylim(ylims[0]-10, ylims[1]+2)


(541.422119140625, 626.08447265625)

In [6]:
# lets take a look at the proportion of approach points before and during the first 30s of the event
start = gt.utils.find_closest(dyad.times, events.start[idx] - 30*dyad.times[1]-dyad.times[0])
stop = gt.utils.find_closest(dyad.times, events.start[idx] + 30*dyad.times[1]-dyad.times[0])

status1 = np.asarray(status1)
status2 = np.asarray(status2)

status1_eventstart = status1[start:stop]
status2_eventstart = status2[start:stop]

plt.plot(dyad.times[start:stop], dyad.fund_id1[start:stop])
plt.plot(dyad.times[start:stop], dyad.fund_id2[start:stop])
plt.plot(dyad.times[start:stop], status1_eventstart + 570, "|")
plt.plot(dyad.times[start:stop], status2_eventstart + 570, "|")


I have the feeling that this will not lead to any cool results because the phases change so fast and there is no clearly visual pattern to the changes of approacher and retreater during events. There could be some pattern, but it is not overly obvious. Maybe filtering or averaging over time or something else would make it more apparent but it looks to noisy to produce meaningful results. 

Lets try somethign else: 
Compute relative heading angle in degrees from the trajectories of the individuals. When heading angles polarize, there is most likely chasing going on. Important is the relative part: If both fish swim in the same direction (i.e. in a trajectory whith the same angle), they could also be swimming into the same direction next to each other. If we calculate the heading angle relative to the position of the other individual, this is not the case. Overall, I expect the relative trajectories to be randomly distributed. During events, they could become polarized or opposing each other, i.e. when they both swim towards each other.

Why do we do this? We can count the number of chasing events during frequency modulations to interpret what the modulations mean. Maybe the number of chasing events changes during modulations, etc.

the trajectory at point t is given by 

$\newcommand\mycolv[1]{\begin{bmatrix}#1\end{bmatrix}}
t_{1} = \mycolv{x_{t+1}\\y_{t+1}} - \mycolv{x_t\\y_t}
$

the angle between fish1 and fish3 is

$
\alpha = \tan^{-1}{\frac{y_{fish2}-y_{fish1}}{x_{fish2}-x_{fish1}}}
$

Now that we have the angle between the fish, we can calculate the relative angle of the trajectory of fish1 to the position of fish2. For this, we just take the angle of the trajectory and subtract it from the angle to the other fish.

$
\beta = \tan^{-1}{\frac{y_{t+1}-y_{t}}{x_{t+1}-x_{t}}}
$

$
\beta_{rel} = \alpha - \beta
$

In [7]:
import math

# keep in mind that atan is in radians!

def rel_angles(dyad, direction):
    if direction == "up":
        xs1 = dyad.xpos_smth_id1[:-1]
        ys1 = dyad.xpos_smth_id1[:-1]
        xs2 = dyad.xpos_smth_id2[:-1]
        ys2 = dyad.xpos_smth_id2[:-1]
    elif direction == "down":
        xs1 = dyad.xpos_smth_id2[:-1]
        ys1 = dyad.xpos_smth_id2[:-1]
        xs2 = dyad.xpos_smth_id1[:-1]
        ys2 = dyad.xpos_smth_id2[:-1]

    angles = [] # absolute angles between individuals
    tangles = [] # angle of id1 trajectory
    rangles = [] # relative angles between individuals

    for i, (x, y) in enumerate(zip(xs1, ys1)):
        
        # angle between:
        if direction == "up":
            dy = dyad.xpos_smth_id2[i+1] - y
            dx = dyad.xpos_smth_id2[i+1] - x
            a = math.degrees(math.atan2(dy, dx)) # angle between fish 1 and fish 2
        elif direction == "down":
            dy = dyad.xpos_smth_id1[i+1] - y
            dx = dyad.xpos_smth_id2[i+1] - x
            a = math.degrees(math.atan2(dy, dx)) # angle between fish 1 and fish 2

        # angle of trajectory:
        ty = ys1[i+1] - y
        tx = xs1[i+1] - x
        b = math.degrees(math.atan2(ty, tx))

        if a < 0:
            a = 360 + a
        if b < 0:
            b = 360 + b

        # relative angle
        brel = a - b

        if brel < 0:
            brel = 360 + brel

        # save
        angles.append(a)
        tangles.append(b)
        rangles.append(brel)

    rangles = gt.utils.nanpad(rangles, position="right", padlen=1)
    return rangles

rangels1 = rel_angles(dyad, "up")
rangels2 = rel_angles(dyad, "down")

fig, ax = plt.subplots(2,1, sharex=True)
ax[0].plot(dyad.times, dyad.fund_id1)
ax[0].plot(dyad.times, dyad.fund_id2)
ax[1].plot(dyad.times, rangles_12)


    

IndexError: index 145074 is out of bounds for axis 0 with size 145074

In [ ]:
start = gt.utils.find_closest(dyad.times, 25675)
stop = gt.utils.find_closest(dyad.times, 25975)
#boolean = status1[start:stop]
#boolean = status1
#boolean[np.isnan(boolean)] = 0
#boolean = boolean.astype(dtype=bool)

event_rangles1 = rangles1[start:stop]
event_rangles2 = rangles2[start:stop]
#event_rangles_12 = rangles_12
#event_rangles_12 = event_rangles_12[boolean]
distribution1 = np.histogram(event_rangles1,bins=60, range=(0,360))[0]
distribution2 = np.histogram(event_rangles2,bins=60, range=(0,360))[0]
theta = np.linspace(0,360,60, endpoint=False)

f = plt.figure()
ax = f.add_subplot(polar=True)
ax.set_theta_zero_location('N'); ax.set_theta_direction(-1)
ax.set_ylim(-6,35)
ax.bar(theta, distribution, edgecolor="black", width = 2*np.pi / 60)

NameError: name 'rangles1' is not defined

In [ ]:
plt.plot(dyad.xpos_smth_id1[start:stop], dyad.ypos_smth_id1[start:stop])
plt.plot(dyad.xpos_smth_id2[start:stop], dyad.ypos_smth_id2[start:stop])

In [ ]:
print(len(event_rangles_12))
print(len(boolean))

33481
145075
